In [78]:
from transformers import GPT2LMHeadModel, GPT2Tokenizer, Trainer, TrainingArguments, DataCollatorWithPadding
from datasets import load_dataset
import torch

In [2]:
dataset = load_dataset("wmt16", "de-en")

In [59]:
model_name = "openai-community/gpt2"
tokenizer = GPT2Tokenizer.from_pretrained(model_name)

c:\Users\21700\.conda\envs\nlp\lib\site-packages\huggingface_hub\file_download.py:1132: FutureWarning: `resume_download` is deprecated and will be removed in version 1.0.0. Downloads always resume when possible. If you want to force a new download, use `force_download=True`.
  warnings.warn(


In [37]:
model = GPT2LMHeadModel.from_pretrained(model_name)
model.resize_token_embeddings(len(tokenizer))

c:\Users\21700\.conda\envs\nlp\lib\site-packages\huggingface_hub\file_download.py:1132: FutureWarning: `resume_download` is deprecated and will be removed in version 1.0.0. Downloads always resume when possible. If you want to force a new download, use `force_download=True`.
  warnings.warn(


In [79]:
tokenizer.pad_token = tokenizer.eos_token
def preprocess(examples):
    inputs = ["sentence to translate:" + ex["en"] + ", you answer please:" + ex["de"] for ex in examples["translation"]]
    model_inputs = tokenizer(inputs, max_length=512, truncation=True, padding='max_length', return_tensors="pt")
    
    labels = model_inputs["input_ids"].clone()
    for i, input_id in enumerate(model_inputs["input_ids"]):
        source_length = tokenizer("sentence to translate: " + examples["translation"][i]["en"] + ", you answer please:", truncation=True, max_length=512, return_tensors="pt")["input_ids"].shape[1]
        labels[i][:source_length].fill_(-100)
    
    model_inputs["labels"] = labels
    return model_inputs

In [81]:
dataset['train']

Dataset({
    features: ['translation'],
    num_rows: 4548885
})

In [86]:
train_size = dataset['train'].num_rows
tokenized_train = dataset['train'].shuffle().select(range(train_size // 100)).map(preprocess, batched=True)

Map:   0%|          | 0/45488 [00:00<?, ? examples/s]

In [83]:
validation_size = dataset['validation'].num_rows
tokenized_valid = dataset['validation'].map(preprocess, batched=True)

Map:   0%|          | 0/4 [00:00<?, ? examples/s]

+ 产生数据只需要运行上述代码即可，因为测试集是不能看到翻译结果的。

In [45]:
# 无需运行
train_size = len(tokenized_train)
train_data = tokenized_train.shuffle().select(range(train_size // 100))

In [89]:
training_args = TrainingArguments(
    output_dir='./results_translation_wmt16',
    evaluation_strategy='epoch',
    learning_rate=5e-5,
    per_device_train_batch_size=4,
    per_device_eval_batch_size=4,
    num_train_epochs=3,
    weight_decay=0.01,
)

# collator = DataCollatorWithPadding(tokenizer=tokenizer, padding=True, max_length=512)

# 创建Trainer实例
trainer = Trainer(
    model=model,
    args=training_args,
    train_dataset=tokenized_train,
    eval_dataset=tokenized_valid,
    tokenizer=tokenizer,
    # data_collator=collator
)

# 开始训练
trainer.train()

  0%|          | 0/34116 [00:00<?, ?it/s]

{'loss': 0.4159, 'grad_norm': 0.6467600464820862, 'learning_rate': 4.926720600304843e-05, 'epoch': 0.04}
{'loss': 0.4161, 'grad_norm': 0.5942320823669434, 'learning_rate': 4.853441200609685e-05, 'epoch': 0.09}
{'loss': 0.4052, 'grad_norm': 0.4753303527832031, 'learning_rate': 4.780161800914527e-05, 'epoch': 0.13}
{'loss': 0.3883, 'grad_norm': 0.44854292273521423, 'learning_rate': 4.7068824012193695e-05, 'epoch': 0.18}
{'loss': 0.3736, 'grad_norm': 0.41471239924430847, 'learning_rate': 4.6336030015242114e-05, 'epoch': 0.22}
{'loss': 0.3625, 'grad_norm': 0.6229807734489441, 'learning_rate': 4.560323601829054e-05, 'epoch': 0.26}
{'loss': 0.3541, 'grad_norm': 0.5606170296669006, 'learning_rate': 4.487044202133897e-05, 'epoch': 0.31}
{'loss': 0.3433, 'grad_norm': 0.6385502815246582, 'learning_rate': 4.413764802438739e-05, 'epoch': 0.35}
{'loss': 0.3356, 'grad_norm': 0.4884601831436157, 'learning_rate': 4.3404854027435806e-05, 'epoch': 0.4}
{'loss': 0.3446, 'grad_norm': 0.41243478655815125, 

  0%|          | 0/1 [00:00<?, ?it/s]

{'eval_loss': 0.2553116977214813, 'eval_runtime': 0.2076, 'eval_samples_per_second': 19.269, 'eval_steps_per_second': 4.817, 'epoch': 1.0}
{'loss': 0.2881, 'grad_norm': 0.3625343441963196, 'learning_rate': 3.314573807011373e-05, 'epoch': 1.01}
{'loss': 0.2722, 'grad_norm': 0.5522125959396362, 'learning_rate': 3.2412944073162155e-05, 'epoch': 1.06}
{'loss': 0.2766, 'grad_norm': 0.7345688939094543, 'learning_rate': 3.168015007621058e-05, 'epoch': 1.1}
{'loss': 0.2673, 'grad_norm': 0.3095338046550751, 'learning_rate': 3.0947356079259e-05, 'epoch': 1.14}
{'loss': 0.262, 'grad_norm': 0.5100145936012268, 'learning_rate': 3.0214562082307425e-05, 'epoch': 1.19}
{'loss': 0.2629, 'grad_norm': 0.5949246287345886, 'learning_rate': 2.9481768085355848e-05, 'epoch': 1.23}
{'loss': 0.2695, 'grad_norm': 0.4245113432407379, 'learning_rate': 2.8748974088404267e-05, 'epoch': 1.28}
{'loss': 0.2625, 'grad_norm': 0.4610216021537781, 'learning_rate': 2.8016180091452694e-05, 'epoch': 1.32}
{'loss': 0.2595, 'gr

  0%|          | 0/1 [00:00<?, ?it/s]

{'eval_loss': 0.22642703354358673, 'eval_runtime': 0.1928, 'eval_samples_per_second': 20.751, 'eval_steps_per_second': 5.188, 'epoch': 2.0}
{'loss': 0.2457, 'grad_norm': 0.6066887378692627, 'learning_rate': 1.629147614022746e-05, 'epoch': 2.02}
{'loss': 0.2424, 'grad_norm': 0.4047863185405731, 'learning_rate': 1.5558682143275882e-05, 'epoch': 2.07}
{'loss': 0.2332, 'grad_norm': 0.4347898066043854, 'learning_rate': 1.4825888146324307e-05, 'epoch': 2.11}
{'loss': 0.2406, 'grad_norm': 0.6219213604927063, 'learning_rate': 1.409309414937273e-05, 'epoch': 2.15}
{'loss': 0.2326, 'grad_norm': 0.6340215802192688, 'learning_rate': 1.3360300152421151e-05, 'epoch': 2.2}
{'loss': 0.2345, 'grad_norm': 0.42575132846832275, 'learning_rate': 1.2627506155469576e-05, 'epoch': 2.24}
{'loss': 0.2335, 'grad_norm': 0.5731749534606934, 'learning_rate': 1.1894712158517999e-05, 'epoch': 2.29}
{'loss': 0.2407, 'grad_norm': 0.42256098985671997, 'learning_rate': 1.116191816156642e-05, 'epoch': 2.33}
{'loss': 0.237

  0%|          | 0/1 [00:00<?, ?it/s]

{'eval_loss': 0.2212127149105072, 'eval_runtime': 0.2086, 'eval_samples_per_second': 19.173, 'eval_steps_per_second': 4.793, 'epoch': 3.0}
{'train_runtime': 20876.2354, 'train_samples_per_second': 6.537, 'train_steps_per_second': 1.634, 'train_loss': 0.27731623729545246, 'epoch': 3.0}


TrainOutput(global_step=34116, training_loss=0.27731623729545246, metrics={'train_runtime': 20876.2354, 'train_samples_per_second': 6.537, 'train_steps_per_second': 1.634, 'total_flos': 3.5656955854848e+16, 'train_loss': 0.27731623729545246, 'epoch': 3.0})

+ 正常训练运行以上代码即可

In [90]:
model

GPT2LMHeadModel(
  (transformer): GPT2Model(
    (wte): Embedding(50257, 768)
    (wpe): Embedding(1024, 768)
    (drop): Dropout(p=0.1, inplace=False)
    (h): ModuleList(
      (0-11): 12 x GPT2Block(
        (ln_1): LayerNorm((768,), eps=1e-05, elementwise_affine=True)
        (attn): GPT2Attention(
          (c_attn): Conv1D()
          (c_proj): Conv1D()
          (attn_dropout): Dropout(p=0.1, inplace=False)
          (resid_dropout): Dropout(p=0.1, inplace=False)
        )
        (ln_2): LayerNorm((768,), eps=1e-05, elementwise_affine=True)
        (mlp): GPT2MLP(
          (c_fc): Conv1D()
          (c_proj): Conv1D()
          (act): NewGELUActivation()
          (dropout): Dropout(p=0.1, inplace=False)
        )
      )
    )
    (ln_f): LayerNorm((768,), eps=1e-05, elementwise_affine=True)
  )
  (lm_head): Linear(in_features=768, out_features=50257, bias=False)
)

In [157]:
def process_test(ex):
    inputs = "sentence to translate:" + ex["translation"]["en"] + ", you answer please:" + ex["tran
    model_inputs = tokenizer(inputs, max_length=512, truncation=True, padding='max_length', return_tensors="pt")
    return model_inputs

In [158]:
dataset['test'][2]

{'translation': {'de': 'Die beiden wollten über die Umsetzung der internationalen Vereinbarung sowie über Teherans destabilisierende Maßnahmen im Nahen Osten sprechen.',
  'en': "The two wanted to talk about the implementation of the international agreement and about Teheran's destabilising activities in the Middle East."}}

In [159]:
test1 = process_test(dataset['test'][2])

In [161]:
# print(test1['attention_mask'])
tokenizer.pad_token_id = tokenizer.eos_token_id
tokenizer.decode(model.generate(inputs=test1['input_ids'].cuda(), attention_mask=test1['attention_mask'].cuda(), max_length=600, num_return_sequences=1)[0])

Setting `pad_token_id` to `eos_token_id`:50256 for open-end generation.
A decoder-only architecture is being used, but right-padding was detected! For correct generation results, please set `padding_side='left'` when initializing the tokenizer.


"sentence to translate:The two wanted to talk about the implementation of the international agreement and about Teheran's destabilising activities in the Middle East., you answer please:Die beiden<|endoftext|><|endoftext|><|endoftext|><|endoftext|><|endoftext|><|endoftext|><|endoftext|><|endoftext|><|endoftext|><|endoftext|><|endoftext|><|endoftext|><|endoftext|><|endoftext|><|endoftext|><|endoftext|><|endoftext|><|endoftext|><|endoftext|><|endoftext|><|endoftext|><|endoftext|><|endoftext|><|endoftext|><|endoftext|><|endoftext|><|endoftext|><|endoftext|><|endoftext|><|endoftext|><|endoftext|><|endoftext|><|endoftext|><|endoftext|><|endoftext|><|endoftext|><|endoftext|><|endoftext|><|endoftext|><|endoftext|><|endoftext|><|endoftext|><|endoftext|><|endoftext|><|endoftext|><|endoftext|><|endoftext|><|endoftext|><|endoftext|><|endoftext|><|endoftext|><|endoftext|><|endoftext|><|endoftext|><|endoftext|><|endoftext|><|endoftext|><|endoftext|><|endoftext|><|endoftext|><|endoftext|><|endoftext

In [168]:
output_1 = model(input_ids=test1['input_ids'].cuda(), attention_mask=test1['attention_mask'].cuda())
print(output_1)
tokenizer.decode(output_1.logits[0].argmax(dim=-1))

CausalLMOutputWithCrossAttentions(loss=None, logits=tensor([[[ -14.4350,  -14.2776,  -16.0555,  ...,  -21.0613,  -22.2633,
           -13.1346],
         [  26.4185,   28.1106,   21.8871,  ...,   17.5783,   14.9548,
            26.1459],
         [  13.3491,   18.9127,   17.4725,  ...,    8.7812,    6.7564,
            17.0571],
         ...,
         [-257.4440, -258.0937, -263.4948,  ..., -292.7445, -306.8278,
          -232.5756],
         [-257.3044, -257.9482, -263.3510,  ..., -292.5395, -306.6185,
          -232.3306],
         [-257.6510, -258.2595, -263.6711,  ..., -292.8014, -306.9175,
          -232.8551]]], device='cuda:0', grad_fn=<UnsafeViewBackward0>), past_key_values=((tensor([[[[-0.9121,  1.9380,  0.3453,  ..., -0.7346,  0.1951,  0.6764],
          [-2.4264,  2.1534,  0.4420,  ..., -0.0218, -0.7510,  2.4287],
          [-1.5054,  2.0477,  0.6969,  ..., -0.3393, -1.0869,  1.1048],
          ...,
          [ 2.2475, -2.3027, -0.3033,  ..., -1.2246,  1.0039, -1.9748],
    

' the: ":Dieatre- to be about the event of U U human on the theilan\'s sonisation destabil. Te west East.<|endoftext|> w please:Die beiden w<|endoftext|><|endoftext|><|endoftext|><|endoftext|><|endoftext|><|endoftext|><|endoftext|><|endoftext|><|endoftext|><|endoftext|><|endoftext|><|endoftext|><|endoftext|><|endoftext|><|endoftext|><|endoftext|><|endoftext|><|endoftext|><|endoftext|><|endoftext|><|endoftext|><|endoftext|><|endoftext|><|endoftext|><|endoftext|><|endoftext|><|endoftext|><|endoftext|><|endoftext|><|endoftext|><|endoftext|><|endoftext|><|endoftext|><|endoftext|><|endoftext|><|endoftext|><|endoftext|><|endoftext|><|endoftext|><|endoftext|><|endoftext|><|endoftext|><|endoftext|><|endoftext|><|endoftext|><|endoftext|><|endoftext|><|endoftext|><|endoftext|><|endoftext|><|endoftext|><|endoftext|><|endoftext|><|endoftext|><|endoftext|><|endoftext|><|endoftext|><|endoftext|><|endoftext|><|endoftext|><|endoftext|><|endoftext|><|endoftext|><|endoftext|><|endoftext|><|endoftext|><|

In [134]:
tokenizer.pad_token_id = tokenizer.eos_token_id
input = tokenizer('please translate \'Obama empfängt Netanyahu\' from german to english,', return_tensors='pt')
print(input)
tokenizer.decode(model.generate(inputs = input['input_ids'].cuda(), attention_mask = input['attention_mask'].cuda(), max_length=600, num_return_sequences=1)[0])

Setting `pad_token_id` to `eos_token_id`:50256 for open-end generation.


{'input_ids': tensor([[29688, 15772,   705, 15948,   795,    79,    69, 11033,   782,    83,
         17104,     6,   422,   308,  2224,   284, 46932,    11]]), 'attention_mask': tensor([[1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1]])}


"please translate 'Obama empfängt Netanyahu' from german to english, das ist einmal einmal.<|endoftext|>"

In [169]:
model.save_pretrained('model_translation_wmt16')

+ 以上为测试

In [75]:
def preprocess_single(ex):
    inputs = "sentence to translate:" + ex["translation"]["en"] + ", you answer please:" + ex["translation"]["de"]
    model_inputs = tokenizer(inputs, max_length=512, truncation=True, padding='max_length', return_tensors="pt")
    
    labels = model_inputs["input_ids"].clone()
    # for i, input_id in enumerate(model_inputs["input_ids"]):
    source_length = tokenizer("sentence to translate: " + ex["translation"]["en"] + ", you answer please:", truncation=True, max_length=512, return_tensors="pt")["input_ids"].shape[1]
    # print(tokenizer("translate: " + ex["translation"]["en"] + ", you answer please: ", truncation=True, max_length=512, return_tensors="pt")["input_ids"].shape)
    print(source_length)
    labels[:,:source_length].fill_(-100)
    model_inputs["labels"] = labels
    return model_inputs

In [76]:
tmp = dataset['train'][0]
tmp
tmp = preprocess_single(tmp)
tmp

15


{'input_ids': tensor([[34086,   594,   284, 15772,    25,  4965, 24098,   286,   262,  6246,
            11,   345,  3280,  3387,    25,    54,   798,   263,   559, 22184,
           993,  1326,  4587,   311,  4224,  2150,    82,   525,    72,  1098,
         50256, 50256, 50256, 50256, 50256, 50256, 50256, 50256, 50256, 50256,
         50256, 50256, 50256, 50256, 50256, 50256, 50256, 50256, 50256, 50256,
         50256, 50256, 50256, 50256, 50256, 50256, 50256, 50256, 50256, 50256,
         50256, 50256, 50256, 50256, 50256, 50256, 50256, 50256, 50256, 50256,
         50256, 50256, 50256, 50256, 50256, 50256, 50256, 50256, 50256, 50256,
         50256, 50256, 50256, 50256, 50256, 50256, 50256, 50256, 50256, 50256,
         50256, 50256, 50256, 50256, 50256, 50256, 50256, 50256, 50256, 50256,
         50256, 50256, 50256, 50256, 50256, 50256, 50256, 50256, 50256, 50256,
         50256, 50256, 50256, 50256, 50256, 50256, 50256, 50256, 50256, 50256,
         50256, 50256, 50256, 50256, 5

In [22]:
tokenizer.decode(tmp["input_ids"][0][:15])

'sentence to translate:Resumption of the session, you answer please:'

In [23]:
tmp['labels']

tensor([[ -100,  -100,  -100,  -100,  -100,  -100,  -100,  -100,  -100,  -100,
          -100,  -100,  -100,  -100,  -100,    54,   798,   263,   559, 22184,
           993,  1326,  4587,   311,  4224,  2150,    82,   525,    72,  1098]])

In [27]:
tmp

{'input_ids': tensor([[34086,   594,   284, 15772,    25,  4965, 24098,   286,   262,  6246,
            11,   345,  3280,  3387,    25,    54,   798,   263,   559, 22184,
           993,  1326,  4587,   311,  4224,  2150,    82,   525,    72,  1098]]), 'attention_mask': tensor([[1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1,
         1, 1, 1, 1, 1, 1]]), 'labels': tensor([[ -100,  -100,  -100,  -100,  -100,  -100,  -100,  -100,  -100,  -100,
          -100,  -100,  -100,  -100,  -100,    54,   798,   263,   559, 22184,
           993,  1326,  4587,   311,  4224,  2150,    82,   525,    72,  1098]])}

In [28]:
output = model(**tmp)

In [29]:
output['logits'].shape

torch.Size([1, 30, 50257])

In [30]:
output.logits[0].argmax(dim=1)

tensor([  13,   13,  262,  262,  366,  806,  286,  262, 1175,   13,  290,  389,
         262,   13,   40,  539,  263,  325,   69, 2150,   81,   11,  370, 9116,
        2150,  268, 3212, 3681,  861,   11])

In [33]:
output.loss
tokenizer.decode(output.logits[0].argmax(dim=1))

'.. the the "pect of the war. and are the.Iakeersefungr, Wüungengesimentert,'

In [34]:
dataset['train'][0]['translation']['de']

'Wiederaufnahme der Sitzungsperiode'

In [79]:
print(tokenizer.decode(tmp['labels'][10:]))

In [ ]:
tokenized_datasets = dataset.map(preprocess_function, batched=True)

In [ ]:
"sentence to translate:The two wanted to talk about the implementation of the international agreement and about Teheran's destabilising activities in the Middle East., you answer please:Die beiden<|endoftext|><|endoftext|><|endoftext|><|endoftext|><|endoftext|><|endoftext|><|endoftext|><|endoftext|><|endoftext|><|endoftext|><|endoftext|><|endoftext|><|endoftext|><|endoftext|><|endoftext|><|endoftext|><|endoftext|><|endoftext|><|endoftext|><|endoftext|><|endoftext|><|endoftext|><|endoftext|><|endoftext|><|endoftext|><|endoftext|><|endoftext|><|endoftext|><|endoftext|><|endoftext|><|endoftext|><|endoftext|><|endoftext|><|endoftext|><|endoftext|><|endoftext|><|endoftext|><|endoftext|><|endoftext|><|endoftext|><|endoftext|><|endoftext|><|endoftext|><|endoftext|><|endoftext|><|endoftext|><|endoftext|><|endoftext|><|endoftext|><|endoftext|><|endoftext|><|endoftext|><|endoftext|><|endoftext|><|endoftext|><|endoftext|><|endoftext|><|endoftext|><|endoftext|><|endoftext|><|endoftext|><|endoftext|><|endoftext|><|endoftext|><|endoftext|><|endoftext|><|endoftext|><|endoftext|><|endoftext|><|endoftext|><|endoftext|><|endoftext|><|endoftext|><|endoftext|><|endoftext|><|endoftext|><|endoftext|><|endoftext|><|endoftext|><|endoftext|><|endoftext|><|endoftext|><|endoftext|><|endoftext|><|endoftext|><|endoftext|><|endoftext|><|endoftext|><|endoftext|><|endoftext|><|endoftext|><|endoftext|><|endoftext|><|endoftext|><|endoftext|><|endoftext|><|endoftext|><|endoftext|><|endoftext|><|endoftext|><|endoftext|><|endoftext|><|endoftext|><|endoftext|><|endoftext|><|endoftext|><|endoftext|><|endoftext|><|endoftext|><|endoftext|><|endoftext|><|endoftext|><|endoftext|><|endoftext|><|endoftext|><|endoftext|><|endoftext|><|endoftext|><|endoftext|><|endoftext|><|endoftext|><|endoftext|><|endoftext|><|endoftext|><|endoftext|><|endoftext|><|endoftext|><|endoftext|><|endoftext|><|endoftext|><|endoftext|><|endoftext|><|endoftext|><|endoftext|><|endoftext|><|endoftext|><|endoftext|><|endoftext|><|endoftext|><|endoftext|><|endoftext|><|endoftext|><|endoftext|><|endoftext|><|endoftext|><|endoftext|><|endoftext|><|endoftext|><|endoftext|><|endoftext|><|endoftext|><|endoftext|><|endoftext|><|endoftext|><|endoftext|><|endoftext|><|endoftext|><|endoftext|><|endoftext|><|endoftext|><|endoftext|><|endoftext|><|endoftext|><|endoftext|><|endoftext|><|endoftext|><|endoftext|><|endoftext|><|endoftext|><|endoftext|><|endoftext|><|endoftext|><|endoftext|><|endoftext|><|endoftext|><|endoftext|><|endoftext|><|endoftext|><|endoftext|><|endoftext|><|endoftext|><|endoftext|><|endoftext|><|endoftext|><|endoftext|><|endoftext|><|endoftext|><|endoftext|><|endoftext|><|endoftext|><|endoftext|><|endoftext|><|endoftext|><|endoftext|><|endoftext|><|endoftext|><|endoftext|><|endoftext|><|endoftext|><|endoftext|><|endoftext|><|endoftext|><|endoftext|><|endoftext|><|endoftext|><|endoftext|><|endoftext|><|endoftext|><|endoftext|><|endoftext|><|endoftext|><|endoftext|><|endoftext|><|endoftext|><|endoftext|><|endoftext|><|endoftext|><|endoftext|><|endoftext|><|endoftext|><|endoftext|><|endoftext|><|endoftext|><|endoftext|><|endoftext|><|endoftext|><|endoftext|><|endoftext|><|endoftext|><|endoftext|><|endoftext|><|endoftext|><|endoftext|><|endoftext|><|endoftext|><|endoftext|><|endoftext|><|endoftext|><|endoftext|><|endoftext|><|endoftext|><|endoftext|><|endoftext|><|endoftext|><|endoftext|><|endoftext|><|endoftext|><|endoftext|><|endoftext|><|endoftext|><|endoftext|><|endoftext|><|endoftext|><|endoftext|><|endoftext|><|endoftext|><|endoftext|><|endoftext|><|endoftext|><|endoftext|><|endoftext|><|endoftext|><|endoftext|><|endoftext|><|endoftext|><|endoftext|><|endoftext|><|endoftext|><|endoftext|><|endoftext|><|endoftext|><|endoftext|><|endoftext|><|endoftext|><|endoftext|><|endoftext|><|endoftext|><|endoftext|><|endoftext|><|endoftext|><|endoftext|><|endoftext|><|endoftext|><|endoftext|><|endoftext|><|endoftext|><|endoftext|><|endoftext|><|endoftext|><|endoftext|><|endoftext|><|endoftext|><|endoftext|><|endoftext|><|endoftext|><|endoftext|><|endoftext|><|endoftext|><|endoftext|><|endoftext|><|endoftext|><|endoftext|><|endoftext|><|endoftext|><|endoftext|><|endoftext|><|endoftext|><|endoftext|><|endoftext|><|endoftext|><|endoftext|><|endoftext|><|endoftext|><|endoftext|><|endoftext|><|endoftext|><|endoftext|><|endoftext|><|endoftext|><|endoftext|><|endoftext|><|endoftext|><|endoftext|><|endoftext|><|endoftext|><|endoftext|><|endoftext|><|endoftext|><|endoftext|><|endoftext|><|endoftext|><|endoftext|><|endoftext|><|endoftext|><|endoftext|><|endoftext|><|endoftext|><|endoftext|><|endoftext|><|endoftext|><|endoftext|><|endoftext|><|endoftext|><|endoftext|><|endoftext|><|endoftext|><|endoftext|><|endoftext|><|endoftext|><|endoftext|><|endoftext|><|endoftext|><|endoftext|><|endoftext|><|endoftext|><|endoftext|><|endoftext|><|endoftext|><|endoftext|><|endoftext|><|endoftext|><|endoftext|><|endoftext|><|endoftext|><|endoftext|><|endoftext|><|endoftext|><|endoftext|><|endoftext|><|endoftext|><|endoftext|><|endoftext|><|endoftext|><|endoftext|><|endoftext|><|endoftext|><|endoftext|><|endoftext|><|endoftext|><|endoftext|><|endoftext|><|endoftext|><|endoftext|><|endoftext|><|endoftext|><|endoftext|><|endoftext|><|endoftext|><|endoftext|><|endoftext|><|endoftext|><|endoftext|><|endoftext|><|endoftext|><|endoftext|><|endoftext|><|endoftext|><|endoftext|><|endoftext|><|endoftext|><|endoftext|><|endoftext|><|endoftext|><|endoftext|><|endoftext|><|endoftext|><|endoftext|><|endoftext|><|endoftext|><|endoftext|><|endoftext|><|endoftext|><|endoftext|><|endoftext|><|endoftext|><|endoftext|><|endoftext|><|endoftext|><|endoftext|><|endoftext|><|endoftext|><|endoftext|><|endoftext|><|endoftext|><|endoftext|><|endoftext|><|endoftext|><|endoftext|><|endoftext|><|endoftext|><|endoftext|><|endoftext|><|endoftext|><|endoftext|><|endoftext|><|endoftext|><|endoftext|><|endoftext|><|endoftext|><|endoftext|><|endoftext|><|endoftext|><|endoftext|><|endoftext|><|endoftext|><|endoftext|><|endoftext|><|endoftext|><|endoftext|><|endoftext|><|endoftext|><|endoftext|><|endoftext|><|endoftext|><|endoftext|><|endoftext|><|endoftext|><|endoftext|><|endoftext|><|endoftext|><|endoftext|><|endoftext|><|endoftext|><|endoftext|><|endoftext|><|endoftext|><|endoftext|><|endoftext|><|endoftext|><|endoftext|><|endoftext|><|endoftext|><|endoftext|><|endoftext|><|endoftext|>"
